# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 12 2022 8:30AM,241584,10,0085851660,ISDIN Corporation,Released
1,Jul 12 2022 8:30AM,241584,10,0085851661,ISDIN Corporation,Released
2,Jul 12 2022 8:30AM,241584,10,0085851666,ISDIN Corporation,Released
3,Jul 12 2022 8:30AM,241584,10,0085851668,ISDIN Corporation,Released
4,Jul 12 2022 8:30AM,241584,10,0085851678,ISDIN Corporation,Released
5,Jul 12 2022 8:30AM,241584,10,0085851676,ISDIN Corporation,Released
6,Jul 12 2022 8:30AM,241584,10,0085851686,ISDIN Corporation,Released
7,Jul 12 2022 8:30AM,241584,10,0085851684,ISDIN Corporation,Released
8,Jul 12 2022 8:30AM,241584,10,0085851689,ISDIN Corporation,Released
9,Jul 12 2022 8:30AM,241584,10,0085851692,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,241584,Released,30
52,241585,Released,1
53,241586,Released,1
54,241588,Released,1
55,241589,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241584,NaN,30.0
241585,NaN,1.0
241586,NaN,1.0
241588,NaN,1.0
241589,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241468,0.0,1.0
241479,13.0,4.0
241481,1.0,0.0
241485,0.0,50.0
241486,0.0,50.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241468,0,1
241479,13,4
241481,1,0
241485,0,50
241486,0,50


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241468,0,1
1,241479,13,4
2,241481,1,0
3,241485,0,50
4,241486,0,50


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241468,,1
1,241479,13,4
2,241481,1,
3,241485,,50
4,241486,,50


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 12 2022 8:30AM,241584,10,ISDIN Corporation
30,Jul 12 2022 8:30AM,241588,10,Yusen – 3D Matrix
31,Jul 12 2022 8:30AM,241585,10,"Citieffe, Inc."
32,Jul 12 2022 8:30AM,241586,10,"Citieffe, Inc."
33,Jul 11 2022 3:44PM,241589,19,"AdvaGen Pharma, Ltd"
34,Jul 11 2022 3:27PM,241583,10,ISDIN Corporation
49,Jul 11 2022 3:23PM,241582,10,"Snap Medical Industries, LLC"
59,Jul 11 2022 3:18PM,241581,10,Yusen – 3D Matrix
60,Jul 11 2022 3:15PM,241580,19,Eli Lilly and Company
61,Jul 11 2022 3:11PM,241579,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 12 2022 8:30AM,241584,10,ISDIN Corporation,,30
1,Jul 12 2022 8:30AM,241588,10,Yusen – 3D Matrix,,1
2,Jul 12 2022 8:30AM,241585,10,"Citieffe, Inc.",,1
3,Jul 12 2022 8:30AM,241586,10,"Citieffe, Inc.",,1
4,Jul 11 2022 3:44PM,241589,19,"AdvaGen Pharma, Ltd",,1
5,Jul 11 2022 3:27PM,241583,10,ISDIN Corporation,,15
6,Jul 11 2022 3:23PM,241582,10,"Snap Medical Industries, LLC",,10
7,Jul 11 2022 3:18PM,241581,10,Yusen – 3D Matrix,,1
8,Jul 11 2022 3:15PM,241580,19,Eli Lilly and Company,,1
9,Jul 11 2022 3:11PM,241579,10,Yusen – 3D Matrix,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 12 2022 8:30AM,241584,10,ISDIN Corporation,30,
1,Jul 12 2022 8:30AM,241588,10,Yusen – 3D Matrix,1,
2,Jul 12 2022 8:30AM,241585,10,"Citieffe, Inc.",1,
3,Jul 12 2022 8:30AM,241586,10,"Citieffe, Inc.",1,
4,Jul 11 2022 3:44PM,241589,19,"AdvaGen Pharma, Ltd",1,
5,Jul 11 2022 3:27PM,241583,10,ISDIN Corporation,15,
6,Jul 11 2022 3:23PM,241582,10,"Snap Medical Industries, LLC",10,
7,Jul 11 2022 3:18PM,241581,10,Yusen – 3D Matrix,1,
8,Jul 11 2022 3:15PM,241580,19,Eli Lilly and Company,1,
9,Jul 11 2022 3:11PM,241579,10,Yusen – 3D Matrix,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 12 2022 8:30AM,241584,10,ISDIN Corporation,30,
1,Jul 12 2022 8:30AM,241588,10,Yusen – 3D Matrix,1,
2,Jul 12 2022 8:30AM,241585,10,"Citieffe, Inc.",1,
3,Jul 12 2022 8:30AM,241586,10,"Citieffe, Inc.",1,
4,Jul 11 2022 3:44PM,241589,19,"AdvaGen Pharma, Ltd",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 12 2022 8:30AM,241584,10,ISDIN Corporation,30.0,NaN
1,Jul 12 2022 8:30AM,241588,10,Yusen – 3D Matrix,1.0,NaN
2,Jul 12 2022 8:30AM,241585,10,"Citieffe, Inc.",1.0,NaN
3,Jul 12 2022 8:30AM,241586,10,"Citieffe, Inc.",1.0,NaN
4,Jul 11 2022 3:44PM,241589,19,"AdvaGen Pharma, Ltd",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 12 2022 8:30AM,241584,10,ISDIN Corporation,30.0,0.0
1,Jul 12 2022 8:30AM,241588,10,Yusen – 3D Matrix,1.0,0.0
2,Jul 12 2022 8:30AM,241585,10,"Citieffe, Inc.",1.0,0.0
3,Jul 12 2022 8:30AM,241586,10,"Citieffe, Inc.",1.0,0.0
4,Jul 11 2022 3:44PM,241589,19,"AdvaGen Pharma, Ltd",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,9661680,728.0,16.0
16,482998,1.0,1.0
19,966309,15.0,13.0
20,966241,53.0,10.0
21,241564,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,9661680,728.0,16.0
1,16,482998,1.0,1.0
2,19,966309,15.0,13.0
3,20,966241,53.0,10.0
4,21,241564,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,728.0,16.0
1,16,1.0,1.0
2,19,15.0,13.0
3,20,53.0,10.0
4,21,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,728.0
1,16,Released,1.0
2,19,Released,15.0
3,20,Released,53.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Executing,16.0,1.0,13.0,10.0,1.0
Released,728.0,1.0,15.0,53.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Executing,16.0,1.0,13.0,10.0,1.0
1,Released,728.0,1.0,15.0,53.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Executing,16.0,1.0,13.0,10.0,1.0
1,Released,728.0,1.0,15.0,53.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()